# Import Statements

In [1]:
!pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from chart_studio import plotly
import chart_studio.plotly as py
from tqdm import tqdm
%matplotlib inline
warnings.filterwarnings("ignore")
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from numpy.random import seed
tf.random.set_seed(1)
seed(1)

# MAIN

In [ ]:
data = pd.read_csv('./Cleaned_data.csv')
data.shape

(1624492, 6)

In [ ]:
res = data[pd.to_numeric(data['start_station_id'], errors='coerce').notnull()]
res['start_station_id'] = res['start_station_id'].astype(float)


In [ ]:
res

,start_station_id,started_at,start_station_name,start_lat,start_lng,Rides
0,3138.09,2021-02-04,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,2
1,3138.09,2021-02-05,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
2,3138.09,2021-02-06,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,2
3,3138.09,2021-02-10,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
4,3138.09,2021-02-11,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
...,...,...,...,...,...,...
1624459,8841.03,2022-07-27,W Mosholu Pkwy S & Sedgwick Ave,40.882260,-73.887020,5
1624460,8841.03,2022-07-28,W Mosholu Pkwy S & Sedgwick Ave,40.882240,-73.886968,5
1624461,8841.03,2022-07-29,W Mosholu Pkwy S & Sedgwick Ave,40.882260,-73.887020,1
1624462,8841.03,2022-07-30,W Mosholu Pkwy S & Sedgwick Ave,40.882152,-73.886971,4


In [ ]:
res=res.sort_values(by=["start_station_id"])

In [ ]:
res=res.iloc[0:700000]

In [ ]:
res1=res[["start_station_id","start_station_name","start_lat","start_lng"]]

In [ ]:
res2=res1.drop_duplicates(subset=['start_station_id'])

In [ ]:
res2.to_csv("data_file.csv")

In [ ]:
data[data["start_station_id"]=="2733.03"].head()

,start_station_id,started_at,start_station_name,start_lat,start_lng,Rides
50016,2733.03,2022-03-03,67 St & Erik Pl,40.633385,-74.016562,1
50017,2733.03,2022-03-04,67 St & Erik Pl,40.633385,-74.016562,1
50018,2733.03,2022-03-05,67 St & Erik Pl,40.633385,-74.016562,1
50019,2733.03,2022-03-06,67 St & Erik Pl,40.633385,-74.016562,4
50020,2733.03,2022-03-07,67 St & Erik Pl,40.633385,-74.016562,3


In [ ]:
station_id_map = {data.iloc[idx, 0]:data.iloc[idx, 2] for idx in range(data.shape[0])}
station_id_map

{'3138.09': '58 St & 2 Ave',
 '3161.02': '46 St & 6 Ave',
 '3178.02': '57 St & 1 Ave',
 '3233.06': '44 St & 6 Ave',
 '3306.03': '5 Ave & 41 St',
 '3347.03': '41 St & 4 Ave',
 '3356.05': '44 St & 2 Ave',
 '3379.03': '5 Ave & 37 St',
 '3388.02': '41 St & 3 Ave',
 '3394.03': 'McDonald Ave & Vanderbilt St',
 '3426.02': 'Prospect Ave & Vanderbilt St',
 '3428.02': '42 St & 1 Ave',
 '3451.01': '34 St & 5 Ave',
 '3460.01': '2 Ave & 36 St',
 '3460.04': '2 Av & 37 St',
 '3492.04': '34 St & 4 Ave',
 '3501.01': '39 St & 2 Ave',
 '3507.04': 'Prospect Park W & 20 St',
 '3523.06': '29 St & 5 Ave',
 '3532.02': '2 Ave & 32 St',
 '3538.09': '11 Ave & Prospect Ave',
 '3555.04': '24 St & 5 Ave',
 '3564.04': '30 St & 4 Ave',
 '3588.01': '7 Ave & 20 St',
 '3620.02': 'Windsor Pl & 8 Ave',
 '3628.11': '6 Ave & 21 St',
 '3651.04': 'West Drive & Prospect Park West',
 '3665.06': 'Sterling St & Bedford Ave',
 '3668.04': '21 St & 4 Ave',
 '3696.05': 'Rogers Ave & Sterling St',
 '3699.07': '17 St & 5 Ave',
 '3704.0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1624492 entries, 0 to 1624491
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   start_station_id    1624492 non-null  object 
 1   started_at          1624492 non-null  object 
 2   start_station_name  1624492 non-null  object 
 3   start_lat           1624492 non-null  float64
 4   start_lng           1624492 non-null  float64
 5   Rides               1624492 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 74.4+ MB


In [ ]:
data['start_station_id'] = data['start_station_id'].astype(str)
data['started_at'] = pd.to_datetime(data['started_at'])

In [ ]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
epochs = 50
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [ ]:
# get station IDs with at least 100 rides

check_df = data.groupby('start_station_id').agg({
    'started_at': pd.Series.nunique,
}).reset_index()

station_ids = check_df[check_df['started_at'] >= 100].sort_values(by='start_station_id')['start_station_id'].unique()
station_ids

array(['2733.03', '2782.02', '2832.03', ..., '8841.03', 'SYS032',
       'SYS038'], dtype=object)

In [ ]:
data

,start_station_id,started_at,start_station_name,start_lat,start_lng,Rides
0,3138.09,2021-02-04,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,2
1,3138.09,2021-02-05,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
2,3138.09,2021-02-06,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,2
3,3138.09,2021-02-10,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
4,3138.09,2021-02-11,58 St & 2 Ave - Brooklyn Army Terminal Vaccina...,40.644512,-74.021506,1
...,...,...,...,...,...,...
1624487,SYS038,2022-07-19,Morgan Loading Docks,40.709738,-73.931469,1
1624488,SYS038,2022-07-20,Morgan Loading Docks,40.709724,-73.931520,1
1624489,SYS038,2022-07-21,Morgan Loading Docks,40.709678,-73.931645,1
1624490,SYS038,2022-07-22,Morgan Loading Docks,40.709577,-73.931624,1


In [ ]:

# convert the 'Date' column to datetime format
data['started_at']= pd.to_datetime(data['started_at'])

In [ ]:
for id in station_ids:
    print(f'Station ID: {id}')
    test_df = data[data['start_station_id'] == id]
    # print(f'Datafram Shape: {test_df.shape}')

    df = test_df.groupby('started_at').agg({
        'Rides': 'sum',
        }).reset_index()
    df.sort_values(by='started_at', inplace=True)

    # display(df.head())

    max_size = df.shape[0]
    test_size = 0.10

    idx = int(max_size-(max_size*test_size))

    train = df.iloc[0:idx,:]
    test = df.iloc[idx:,:]

    # print(f'Train Set Size: {train.shape}')
    # print(f'Test Set Size: {test.shape}')

    # print('Min date from train set: %s' % train['started_at'].min().date())
    # print('Max date from train set: %s' % train['started_at'].max().date())

    # print('Min date from train set: %s' % test['started_at'].min().date())
    # print('Max date from train set: %s' % test['started_at'].max().date())

    lag_size = (test['started_at'].max().date() - train['started_at'].max().date()).days
    # print('Forecast lag size: ', lag_size)

    train.set_index('started_at', inplace=True)

    window = 29
    lag = lag_size
    series = series_to_supervised(train, window=window, lag=lag)

    labels_col = 'Rides(t+%d)' % lag_size
    labels = series[labels_col]
    series = series.drop(labels_col, axis=1)

    test_split_size = 0.1
    X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=test_split_size, shuffle=False)


    X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))

    subsequences = 2
    timesteps = X_train_series.shape[1]//subsequences
    X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, timesteps, 1))
    X_valid_series_sub = X_valid_series.reshape((X_valid_series.shape[0], subsequences, timesteps, 1))


    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(50, activation='relu'))
    model_cnn_lstm.add(Dense(128))
    model_cnn_lstm.add(Dense(64))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss='mse')

    cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, Y_train, validation_data=(X_valid_series_sub, Y_valid), epochs=epochs, verbose=0)

    # fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(12,6))
    # ax1 = axes

    # ax1.plot(cnn_lstm_history.history['loss'], label='Train loss')
    # ax1.plot(cnn_lstm_history.history['val_loss'], label='Validation loss')
    # ax1.legend(loc='best')
    # ax1.set_title('CNN-LSTM')
    # ax1.set_xlabel('Epochs')
    # ax1.set_ylabel('MSE')
    # plt.show()


    cnn_lstm_train_pred = model_cnn_lstm.predict(X_train_series_sub)
    cnn_lstm_valid_pred = model_cnn_lstm.predict(X_valid_series_sub)
    cnn_lstm_valid_pred = np.rint(cnn_lstm_valid_pred[:,0])
    print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cnn_lstm_train_pred)))
    print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, cnn_lstm_valid_pred)))


    train_idx = X_train.index.tolist()
    val_idx = X_valid.index.tolist()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_idx[::3], y=Y_train[::3], mode='lines', name='true'))
    fig.add_trace(go.Scatter(x=val_idx[::3], y=Y_valid[::3], mode='lines', name='val'))
    fig.add_trace(go.Scatter(x=val_idx[::3], y=cnn_lstm_valid_pred[::3], mode='lines', name='pred'))

    fig.update_layout(
        title=f"Forecast Plot for Station ID: {station_id_map[id]}",
        xaxis_title="Date",
        yaxis_title="Number of Rides",
        legend_title="Line Plot",
        hovermode="x unified"
    )
    fig.show()

    fig.write_html(f'./Station_ID_{id}_forecast.html')

    output_train_df = pd.DataFrame(list(zip(train_idx, Y_train, cnn_lstm_train_pred)), columns=['Date', 'True', 'Predicted'])
    output_valid_df = pd.DataFrame(list(zip(val_idx, Y_valid, cnn_lstm_valid_pred)), columns=['Date', 'True', 'Predicted'])

    print('*'*150)
    break

Station ID: 2733.03
2/2 [==============================] - 0s 8ms/step
Train rmse: 3.6542445515072237
Validation rmse: 5.538562600140552


******************************************************************************************************************************************************
